## Resistances

We are missing the resistances, lets fetch them quickly.
We know the kaggle data we are using comes from the scrapping of the roll20 webpage. It seems however they missed/ignored some of the data from the webpage. We will scrap them to complete our database.

In [3]:
#imports
import pandas as pd
import requests
from bs4 import BeautifulSoup

In [23]:
df = pd.read_csv("monsters.csv")
df.head(5)

,Unnamed: 0,name,size,monster_type,ac,hp,str,str_mod,dex,dex_mod,...,dex_save,con_save,int_save,wis_save,cha_save,speed,swim,fly,climb,numeric_cr
0,0,Aboleth,Large,Aberration,17,135,21,5,9,-1,...,0,6,8,6,0,10,40,0,0,10.00
1,1,Acolyte,Medium,Humanoid (any race),10,9,10,0,10,0,...,0,0,0,0,0,30,0,0,0,0.25
2,2,Adult Black Dragon,Huge,Dragon,19,195,23,6,14,2,...,7,10,0,6,8,40,40,80,0,14.00
3,3,Adult Blue Dragon,Huge,Dragon,19,225,25,7,10,0,...,5,11,0,7,9,40,0,80,0,16.00
4,4,Adult Brass Dragon,Huge,Dragon,18,172,23,6,10,0,...,5,10,0,6,8,40,0,80,0,13.00


In [24]:
df["FORCE_res"] = 0
df["THUNDER_res"] = 0
df["RADIANT_res"] = 0
df["FORCE_immun"] = 0
df["THUNDER_immun"] = 0
df["RADIANT_immun"] = 0

In [25]:
url = "https://roll20.net/compendium/dnd5e/"

In [26]:
#data dictionary for every monster
url_dic = {}
for monster in df.name:
    url_ = url + monster

    page = requests.get(url_)
    soup = BeautifulSoup(page.content, 'html.parser')

    cl = "row attrListItem"
    attrib_found = soup.find_all("div", {"class": cl})

    url_dic[monster] = attrib_found

In [27]:
#application to dataframe
res_link = { "force" : "FORCE_res",
           "thunder" : "THUNDER_res",
           "radiant" : "RADIANT_res"}
imm_link = { "force" : "FORCE_immun",
           "thunder" : "THUNDER_immun",
           "radiant" : "RADIANT_immun"}

for monster in df.name:
    
    attrib_found = url_dic[monster]

    for element in attrib_found:
        #fetching the immunities
        if element.find("div", {"class": "col-md-3 attrName"}).text.lower() == "immunities":
            im = element.find("div", {"class": "value"}).text
            immunities_str = im.lower()
            
            for imm in imm_link.keys():
                if imm in immunities_str:
                    df.loc[(df['name'] == monster), imm_link[imm]] = 1 
                    
        #fetching the resistances
        if element.find("div", {"class": "col-md-3 attrName"}).text.lower() == "resistances":
            re = element.find("div", {"class": "value"}).text
            resistances_str = re.lower()
            
            for res in res_link.keys():
                if res in resistances_str:
                    df.loc[(df['name'] == monster), res_link[res]] = 1 
                    

In [5]:
cols_to_see = ["name", "cr",
              'FORCE_res', 'FORCE_immun', 'RADIANT_res', 
               'RADIANT_immun', 'THUNDER_res', 'THUNDER_immun']

df[(df['FORCE_res'] == 1) |\
           (df['THUNDER_res'] == 1) |\
           (df['RADIANT_res'] == 1) |\
           (df['FORCE_immun'] == 1) |\
           (df['THUNDER_immun'] == 1) |\
           (df['RADIANT_immun'] == 1)][cols_to_see]\
            .style.apply(lambda series: ['background-color: #F8CECA' if value == 1 else '' for value in series], axis=0)


,name,cr,FORCE_res,FORCE_immun,RADIANT_res,RADIANT_immun,THUNDER_res,THUNDER_immun
0,Air Elemental,5,0,0,0,0,1,0
1,Couatl,4,0,0,1,0,0,0
2,Deva,10,0,0,1,0,0,0
3,Djinni,11,0,0,0,0,0,1
4,Ghost,4,0,0,0,0,1,0
5,Planetar,16,0,0,1,0,0,0
6,Shadow,1/2,0,0,0,0,1,0
7,Solar,21,0,0,1,0,0,0
8,Specter,1,0,0,0,0,1,0
9,Storm Giant,13,0,0,0,0,0,1


In [62]:
print("- Total number of resistance and immunities -")

for element in ["Force", "Thunder", "Radiant"]:
    cols = [element.upper() + suffix for suffix in ["_res", "_immun"]]
    print(f"{element}: {df[df[cols].eq(1).any(axis=1)].shape[0]}")

- Total number of resistance and immunities -
Force: 0
Thunder: 8
Radiant: 4
